In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numba import jit
import ar1_approx as ar
import time
from scipy.optimize import root as root
from scipy.optimize import fminbound as fminbound
from math import sqrt
%matplotlib inline

# Bonus Problem

### Parameters

In [8]:
# Household parameters
beta = 0.96
h = 6.616
r = (1/beta)-1
# Firm parameters
alpha_k = 0.297
alpha_l = 0.65
delta = 0.154
psi = 1.08
betafirm = 1/(1+r)
# Parameters regarding shock
sigma_eps = 0.213
mu = 0.0
rho = 0.7605
sizez = 9

### Capital Stock Grid

In [9]:
# Discretizing the space of capital stock 
dens = 1
# put in bounds here for the capital stock space
kstar = ((((1 / betafirm - 1 + delta) * ((0.7 / alpha_l) **
                                         (alpha_l / (1 - alpha_l)))) /
         (alpha_k * (1** (1 / (1 - alpha_l))))) **
         ((1 - alpha_l) / (alpha_k + alpha_l - 1)))
kbar = 2*kstar
lb_k = 0.001
ub_k = kbar
krat = np.log(lb_k / ub_k)
numb = np.ceil(krat / np.log(1 - delta))
K = np.zeros(int(numb * dens))
# we'll create in a way where we pin down the upper bound - since
# the distance will be small near the lower bound, we'll miss that by little
@jit
def create_K(numb,dens,delta,ub_k):
    for j in range(int(numb * dens)):
        K[j] = ub_k * (1 - delta) ** (j / dens)
    return K

K = create_K(numb,dens,delta,ub_k)
kvec = K[::-1]
sizek = kvec.shape[0]

### Productivity Shocks

In [10]:
# Find the Markov process for the shocks
z_grid, pi = ar.addacooper(sizez,mu,rho,sigma_eps)
z_grid = np.exp(z_grid) #make it to ln

In [11]:
@jit
def find_op(w):
    op = np.zeros((sizek,sizez))
    for i in range(len(z_grid)):
        op[:,i] = ((1 - alpha_l) * ((alpha_l / w) ** (alpha_l / (1 - alpha_l))) *
              (z_grid[i] ** (1 / (1 - alpha_l)))*((kvec ** alpha_k) ** (1 / (1 - alpha_l))))
    return op

@jit 
def find_labor(w):
    labor = np.zeros((sizek,sizez))
    for i in range(len(z_grid)):
        labor[:,i] = (((alpha_l / w) ** (1 / (1 - alpha_l))) *
              (z_grid[i] ** (1 / (1 - alpha_l)))*((kvec ** alpha_k) ** (1 / (1 - alpha_l))))
    return labor    

@jit
def find_earnings(op):
    e = np.zeros((sizek,sizek,sizez))
    for i in range(sizek):
        for j in range(sizek):
            for k in range(sizez):
                e[i,j,k] = (op[i,k] - kvec[j] + ((1 - delta) * kvec[i]) -
                           ((psi / 2) * ((kvec[j] - ((1 - delta) * kvec[i])) ** 2)
                            / kvec[i]))
    return e

@jit
def VFI_loop(V,Vmat,e):
    for i in range(sizek):  # loop over k
            for j in range(sizek):  # loop over k'
                for k in range(sizez): #loop over z
                    Vmat[i,j,k] = e[i,j,k] + betafirm * (V[j,:]@pi[k,:])
    return Vmat

@jit
def make_gamma(PF):
    Gamma = np.ones((sizez, sizek)) * (1 / (sizek * sizez))
    SDtol = 1e-12
    SDdist = 7
    SDiter = 0
    SDmaxiter = 1000
    while SDdist > SDtol and SDmaxiter > SDiter:
        HGamma = np.zeros((sizez, sizek))
        for i in range(sizez):  # z
            for j in range(sizek):  # k
                for m in range(sizez):  # z'
                    HGamma[m, PF.T[i, j]] = HGamma[m, PF.T[i, j]] + pi[i, m] * Gamma[i, j]
        SDdist = (np.absolute(HGamma - Gamma)).max()
        Gamma = HGamma
        SDiter += 1
    return Gamma

@jit
def policy_func(PF):
    optK = np.zeros((sizek,sizez))
    for k in range(sizez):
        optK[:,k] = kvec[PF[:,k]]
    # Find policy function for investment
    optI = np.zeros((sizek,sizez))
    for k in range(sizez):
        optI[:,k] = optK[:,k]-(1-delta)*kvec
    return optK,optI

def find_wage(w):
    # Operating Profit
    op = find_op(w)
    # Labor demand
    labor = find_labor(w)
    # Earnings
    e= find_earnings(op)
    # Value Function Iteration
    VFtol = 1e-6
    VFdist = 7.0
    VFmaxiter = 3000
    V = np.zeros((sizek,sizez))  # initial guess at value function for z and k
    Vmat = np.zeros((sizek,sizek,sizez))  # initialize Vmat matrix
    VFiter = 1
    while VFdist > VFtol and VFiter < VFmaxiter:
        TV = np.copy(V)
        Vmat = VFI_loop(V,Vmat,e)
        V = Vmat.max(axis=1)
        PF = np.argmax(Vmat,axis=1)
        VFdist = (np.absolute(V - TV)).max()  
        VFiter += 1
    # Find policy function for capital and investment
    optK, optI = policy_func(PF)
    # Stationary distribution of firms
    Gamma = make_gamma(PF)
    # Aggregate Labor
    Ld = np.sum(labor*Gamma.T)
    # Aggregate Investment
    I = np.sum(optI*Gamma.T)
    # Aggregate Adjustment costs
    costs = np.zeros((sizek,sizez))
    for i in range(len(z_grid)):
        costs[:,i] = ((psi / 2) * ((optK[:,i] - ((1 - delta) * kvec)) ** 2)/ kvec)
    Adj = np.sum(costs*Gamma.T)
    # Aggregate Output
    outputs = np.zeros((sizek,sizez))
    for i in range(len(z_grid)):
        outputs[:,i] = z_grid[i]*(kvec**alpha_l)*((labor[:,i])**alpha_l)
    Y = np.sum(outputs*Gamma.T)
    # Solve for household consumption using aggregate resource constraint
    C = Y - I - Adj
    # Find household's labor supply
    Ls = -(w/(h*C))
    return abs(Ld-Ls)

In [18]:
start_time = time.clock()
SS_wage = root(find_wage,0.8,tol=1e-3)
w_time = time.clock() - start_time
print("The steady state wage is:", SS_wage.x[0])
print("Found in seconds:", w_time)

The steady state wage is: 1.05028579144
Found in seconds: 7.025708000000002
